In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import netCDF4 as ncdf
import matplotlib as mlab
import matplotlib.pyplot as plt
import xarray as xr
import glob as glob
import os as os
import sys as sys
from file_tools import *

# These are 45 vertical levels that the FV3 puts out - use them here to map ARW to that grid for comparison

plevels = np.asarray([100000.,  97500.,  95000.,  92500.,  90000.,  87500.,  85000.,  82500.,
                       80000.,  77500.,  75000.,  72500.,  70000.,  67500.,  65000.,  62500.,
                       60000.,  57500.,  55000.,  52500.,  50000.,  47500.,  45000.,  42500.,
                       40000.,  37500.,  35000.,  32500.,  30000.,  27500.,  25000.,  22500.,
                       20000.,  17500.,  15000.,  12500.,  10000.,   7000.,   5000.,   3000.,
                        2000.,   1000.,    700.,    500.,    200.])

nz_new = plevels.shape[0]

_w_threshold = 5.0
_Rgas       = 287.04
_gravity    = 9.806

# default setting for domain cutting

sw_corner = None
ne_corner = None


In [4]:
#-------------------------------------------------------------------------------------------------------
# Set up file lists for processing
#-------------------------------------------------------------------------------------------------------

date = '2021090100'

# HRRR files

# hrrr_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/HRRR'
# hrrr_files = sorted(glob.glob(hrrr_dir+'/hrrr.*.grib2'))
hrrr_dir = '/work/larissa.reames/%s/hrrr'%date
hrrr_files = sorted(glob.glob(hrrr_dir+'/*.grib2'))

# RRFS1 dir

#rrfs_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/RRFS/ctrl'
#rrfs_files = sorted(glob.glob(rrfs_dir+'/PRSLEV.Grb*'))
rrfs_dir1 = '/work/larissa.reames/%s/ctrl'%date
rrfs_files1 = sorted(glob.glob(rrfs_dir1+'/*.grib2'))

# RRFS2 dir

#rrfs_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/RRFS/ctrl'
#rrfs_files = sorted(glob.glob(rrfs_dir+'/PRSLEV.Grb*'))
rrfs_dir2 = '/work/larissa.reames/%s/nord3'%date
rrfs_files2 = sorted(glob.glob(rrfs_dir2+'/*.grib2'))



In [5]:
#-------------------------------------------------------------------------------------------------------
# Specify regional domain
# This is tricky, you need to know where to put the domain before you can actually see it easily

# One way is to process a single file perhaps when convection is active, and then choose the domain.  
# Ideally, the domains should be nx > 500, ny > 500, e.g., about 12-15 degrees in both dlat and dlon
#-------------------------------------------------------------------------------------------------------
#

if (date == '20210526'):
# May 26 Case (NW KS) domain
    sw_corner = (30.0, -105.0)
    ne_corner = (43.0, -91.0)

# 19 July 2019 (N Plains) domain
elif (date == '2019071918'):
    sw_corner = (38.0, -97.0)
    ne_corner = (50.0, -82.0)
    
# 02 March 2020 (Upper MS Valley) domain
elif (date == '2020030212'):
    sw_corner = (30.0, -93.0)
    ne_corner = (42.0, -80.0)
    
# 03 May 2020 (Central MS Valley) domain
elif (date == '2020050300'):
    sw_corner = (31.0, -96.0)
    ne_corner = (44.0, -81.0)
    
# 07 July 2020 (Central Plains) domain
elif (date == '2020070700'):
    sw_corner = (38.0, -105.0)
    ne_corner = (50.0, -92.0)
    
# 07 July 2020 (Upper MW) domain
elif (date == '2020081006'):
    sw_corner = (35.0, -100.0)
    ne_corner = (48.0, -86.0)

# 01 September 2021 (Mid-Atlantic) domain
elif (date == '2021090100'):
    sw_corner = (32, -82.0)
    ne_corner = (44.0, -69.0)

#-------------------------------------------------------------------------------------------------------
# XXX Case


In [7]:
#-------------------------------------------------------------------------------------------------------
#
# Process the HRRR files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in hrrr_files:
    hrrr_ds, filename = hrrr_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None, interpP=True)

------------------------------------------------------------------------------------------------------------------------ 

HRRR Grib READ: Extracting variables from grib file: /work/larissa.reames/2021090100/hrrr/hrrr.t00z.wrfnatf18.grib2 

InterpP is True, Read 3D pressure field from GRIB file

P-max:  102258.83  P-min:  1731.48

Reading my variable:  TEMP from GRIB file variable: Temperature

Reading my variable:  OMEGA from GRIB file variable: Vertical velocity

Reading my variable:  U from GRIB file variable: U component of wind

Reading my variable:  V from GRIB file variable: V component of wind

Reading my variable:  GPH from GRIB file variable: Geopotential Height

Reading my variable:  UH from GRIB file variable: unknown

Reading my variable:  CREF from GRIB file variable: Maximum/Composite radar reflectivity

Reading my variable:  HGT from GRIB file variable: Orography

Creating a sub-region of grid: 32.00, -82.00, 44.00, -69.00 

Successfully wrote new data to file:: /work/l

KeyboardInterrupt: 

In [6]:
#-------------------------------------------------------------------------------------------------------
#
# Process the RRFS files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in rrfs_files1:
    rrfs_ds1, filename = fv3_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None)

------------------------------------------------------------------------------------------------------------------------ 

RRFS Grib READ: Extracting variables from grib file: /work/larissa.reames/2021090100/ctrl/PRSLEV.GrbF18.grib2 

Reading my variable:  TEMP from GRIB variable: Temperature

k 0 ind 44 Level 1000 obj_level 1000:    Min 280.2      Max 316.6
k 1 ind 43 Level 975 obj_level 975:    Min 278.9      Max 315.1
k 2 ind 42 Level 950 obj_level 950:    Min 277.6      Max 313.5
k 3 ind 41 Level 925 obj_level 925:    Min 276.5      Max 311.9
k 4 ind 40 Level 900 obj_level 900:    Min 275.5      Max 310.3
k 5 ind 39 Level 875 obj_level 875:    Min 274.4      Max 308.7
k 6 ind 38 Level 850 obj_level 850:    Min 273.2      Max 307.3
k 7 ind 37 Level 825 obj_level 825:    Min 272.6      Max 304.9
k 8 ind 36 Level 800 obj_level 800:    Min 272.2      Max 301.7
k 9 ind 35 Level 775 obj_level 775:    Min 271.0      Max 299.2
k 10 ind 34 Level 750 obj_level 750:    Min 269.9      Max 296.

k 32 ind 12 Level 200 obj_level 200:    Min -5.2      Max 43.4
k 33 ind 11 Level 175 obj_level 175:    Min -4.4      Max 44.8
k 34 ind 10 Level 150 obj_level 150:    Min -5.0      Max 44.5
k 35 ind 9 Level 125 obj_level 125:    Min -8.6      Max 40.3
k 36 ind 8 Level 100 obj_level 100:    Min -8.9      Max 22.6
k 37 ind 7 Level 70 obj_level 70:    Min -7.4      Max  3.0
k 38 ind 6 Level 50 obj_level 50:    Min -6.8      Max  3.2
k 39 ind 5 Level 30 obj_level 30:    Min -6.3      Max  3.0
k 40 ind 4 Level 20 obj_level 20:    Min -6.4      Max  2.6
k 41 ind 3 Level 10 obj_level 10:    Min -6.7      Max  3.4
k 42 ind 2 Level 7 obj_level 7:    Min -4.9      Max  2.9
k 43 ind 1 Level 5 obj_level 5:    Min -4.0      Max  2.7
k 44 ind 0 Level 2 obj_level 2:    Min -2.6      Max  1.9
Reading my variable:  U from GRIB variable: U component of wind

k 0 ind 44 Level 1000 obj_level 1000:    Min -17.7      Max 35.3
k 1 ind 43 Level 975 obj_level 975:    Min -19.0      Max 28.3
k 2 ind 42 Level 950

k 18 ind 26 Level 550 obj_level 550:    Min 255.2      Max 276.2
k 19 ind 25 Level 525 obj_level 525:    Min 253.0      Max 274.7
k 20 ind 24 Level 500 obj_level 500:    Min 250.6      Max 273.2
k 21 ind 23 Level 475 obj_level 475:    Min 248.0      Max 271.7
k 22 ind 22 Level 450 obj_level 450:    Min 245.1      Max 269.3
k 23 ind 21 Level 425 obj_level 425:    Min 242.5      Max 266.8
k 24 ind 20 Level 400 obj_level 400:    Min 239.1      Max 264.1
k 25 ind 19 Level 375 obj_level 375:    Min 236.1      Max 261.1
k 26 ind 18 Level 350 obj_level 350:    Min 232.8      Max 258.0
k 27 ind 17 Level 325 obj_level 325:    Min 228.6      Max 254.4
k 28 ind 16 Level 300 obj_level 300:    Min 224.0      Max 250.5
k 29 ind 15 Level 275 obj_level 275:    Min 220.2      Max 246.2
k 30 ind 14 Level 250 obj_level 250:    Min 219.7      Max 241.0
k 31 ind 13 Level 225 obj_level 225:    Min 217.2      Max 236.0
k 32 ind 12 Level 200 obj_level 200:    Min 214.4      Max 230.9
k 33 ind 11 Level 175 obj

k 10 ind 34 Level 750 obj_level 750:    Min -23.1      Max 27.2
k 11 ind 33 Level 725 obj_level 725:    Min -20.4      Max 29.8
k 12 ind 32 Level 700 obj_level 700:    Min -18.3      Max 28.9
k 13 ind 31 Level 675 obj_level 675:    Min -16.5      Max 29.3
k 14 ind 30 Level 650 obj_level 650:    Min -14.5      Max 31.7
k 15 ind 29 Level 625 obj_level 625:    Min -12.4      Max 31.7
k 16 ind 28 Level 600 obj_level 600:    Min -11.7      Max 31.0
k 17 ind 27 Level 575 obj_level 575:    Min -12.5      Max 30.6
k 18 ind 26 Level 550 obj_level 550:    Min -13.7      Max 31.7
k 19 ind 25 Level 525 obj_level 525:    Min -12.8      Max 31.2
k 20 ind 24 Level 500 obj_level 500:    Min -12.8      Max 30.2
k 21 ind 23 Level 475 obj_level 475:    Min -11.5      Max 29.7
k 22 ind 22 Level 450 obj_level 450:    Min -11.2      Max 31.2
k 23 ind 21 Level 425 obj_level 425:    Min -12.1      Max 33.0
k 24 ind 20 Level 400 obj_level 400:    Min -11.5      Max 35.7
k 25 ind 19 Level 375 obj_level 375:    

k 39 ind 5 Level 30 obj_level 30:    Min 213.5      Max 224.6
k 40 ind 4 Level 20 obj_level 20:    Min 217.9      Max 228.8
k 41 ind 3 Level 10 obj_level 10:    Min 224.4      Max 237.2
k 42 ind 2 Level 7 obj_level 7:    Min 229.9      Max 240.2
k 43 ind 1 Level 5 obj_level 5:    Min 235.2      Max 246.6
k 44 ind 0 Level 2 obj_level 2:    Min 244.0      Max 256.3
Reading my variable:  HGT from GRIB variable: Orography

Reading my variable:  GPH from GRIB variable: Geopotential Height

k 0 ind 44 Level 1000 obj_level 1000:    Min -97.1      Max 184.3
k 1 ind 43 Level 975 obj_level 975:    Min 123.9      Max 398.3
k 2 ind 42 Level 950 obj_level 950:    Min 346.4      Max 614.8
k 3 ind 41 Level 925 obj_level 925:    Min 577.7      Max 835.7
k 4 ind 40 Level 900 obj_level 900:    Min 815.2      Max 1061.6
k 5 ind 39 Level 875 obj_level 875:    Min 1058.3      Max 1295.0
k 6 ind 38 Level 850 obj_level 850:    Min 1307.6      Max 1543.7
k 7 ind 37 Level 825 obj_level 825:    Min 1563.7      

KeyboardInterrupt: 

In [6]:
#-------------------------------------------------------------------------------------------------------
#
# Process the RRFS files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in rrfs_files2:
    rrfs_ds2, filename = fv3_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None)

------------------------------------------------------------------------------------------------------------------------ 

RRFS Grib READ: Extracting variables from grib file: /work/larissa.reames/2021090100/nord3/PRSLEV.GrbF18.grib2 

Reading my variable:  TEMP from GRIB variable: Temperature

k 0 ind 44 Level 1000 obj_level 1000:    Min 280.2      Max 316.5
k 1 ind 43 Level 975 obj_level 975:    Min 278.9      Max 315.0
k 2 ind 42 Level 950 obj_level 950:    Min 277.6      Max 313.4
k 3 ind 41 Level 925 obj_level 925:    Min 276.5      Max 311.9
k 4 ind 40 Level 900 obj_level 900:    Min 275.5      Max 310.2
k 5 ind 39 Level 875 obj_level 875:    Min 274.4      Max 308.6
k 6 ind 38 Level 850 obj_level 850:    Min 273.2      Max 307.5
k 7 ind 37 Level 825 obj_level 825:    Min 272.5      Max 304.8
k 8 ind 36 Level 800 obj_level 800:    Min 272.2      Max 301.6
k 9 ind 35 Level 775 obj_level 775:    Min 271.0      Max 299.1
k 10 ind 34 Level 750 obj_level 750:    Min 269.9      Max 296

k 32 ind 12 Level 200 obj_level 200:    Min -5.2      Max 40.9
k 33 ind 11 Level 175 obj_level 175:    Min -6.5      Max 42.3
k 34 ind 10 Level 150 obj_level 150:    Min -4.7      Max 41.7
k 35 ind 9 Level 125 obj_level 125:    Min -7.7      Max 36.9
k 36 ind 8 Level 100 obj_level 100:    Min -8.6      Max 18.3
k 37 ind 7 Level 70 obj_level 70:    Min -5.3      Max  4.6
k 38 ind 6 Level 50 obj_level 50:    Min -3.6      Max  6.3
k 39 ind 5 Level 30 obj_level 30:    Min -4.1      Max  3.7
k 40 ind 4 Level 20 obj_level 20:    Min -5.3      Max  2.9
k 41 ind 3 Level 10 obj_level 10:    Min -6.1      Max  3.4
k 42 ind 2 Level 7 obj_level 7:    Min -4.5      Max  2.9
k 43 ind 1 Level 5 obj_level 5:    Min -3.4      Max  2.0
k 44 ind 0 Level 2 obj_level 2:    Min -3.2      Max  1.4
Reading my variable:  U from GRIB variable: U component of wind

k 0 ind 44 Level 1000 obj_level 1000:    Min -17.2      Max 29.3
k 1 ind 43 Level 975 obj_level 975:    Min -15.9      Max 27.5
k 2 ind 42 Level 950

k 18 ind 26 Level 550 obj_level 550:    Min 255.3      Max 277.5
k 19 ind 25 Level 525 obj_level 525:    Min 252.9      Max 275.8
k 20 ind 24 Level 500 obj_level 500:    Min 250.5      Max 274.0
k 21 ind 23 Level 475 obj_level 475:    Min 247.8      Max 272.2
k 22 ind 22 Level 450 obj_level 450:    Min 245.0      Max 270.3
k 23 ind 21 Level 425 obj_level 425:    Min 242.2      Max 267.7
k 24 ind 20 Level 400 obj_level 400:    Min 239.2      Max 264.4
k 25 ind 19 Level 375 obj_level 375:    Min 236.0      Max 261.8
k 26 ind 18 Level 350 obj_level 350:    Min 232.8      Max 258.9
k 27 ind 17 Level 325 obj_level 325:    Min 228.5      Max 255.6
k 28 ind 16 Level 300 obj_level 300:    Min 224.3      Max 252.1
k 29 ind 15 Level 275 obj_level 275:    Min 220.5      Max 248.2
k 30 ind 14 Level 250 obj_level 250:    Min 219.6      Max 243.4
k 31 ind 13 Level 225 obj_level 225:    Min 217.2      Max 234.4
k 32 ind 12 Level 200 obj_level 200:    Min 214.9      Max 230.9
k 33 ind 11 Level 175 obj

k 10 ind 34 Level 750 obj_level 750:    Min -22.3      Max 26.0
k 11 ind 33 Level 725 obj_level 725:    Min -19.7      Max 26.6
k 12 ind 32 Level 700 obj_level 700:    Min -16.9      Max 27.6
k 13 ind 31 Level 675 obj_level 675:    Min -15.1      Max 28.4
k 14 ind 30 Level 650 obj_level 650:    Min -13.9      Max 30.3
k 15 ind 29 Level 625 obj_level 625:    Min -11.8      Max 30.8
k 16 ind 28 Level 600 obj_level 600:    Min -11.6      Max 29.9
k 17 ind 27 Level 575 obj_level 575:    Min -12.3      Max 30.2
k 18 ind 26 Level 550 obj_level 550:    Min -12.4      Max 30.8
k 19 ind 25 Level 525 obj_level 525:    Min -12.9      Max 29.4
k 20 ind 24 Level 500 obj_level 500:    Min -11.5      Max 29.2
k 21 ind 23 Level 475 obj_level 475:    Min -11.1      Max 29.5
k 22 ind 22 Level 450 obj_level 450:    Min -13.0      Max 30.7
k 23 ind 21 Level 425 obj_level 425:    Min -12.8      Max 33.5
k 24 ind 20 Level 400 obj_level 400:    Min -12.3      Max 36.5
k 25 ind 19 Level 375 obj_level 375:    

k 42 ind 2 Level 7 obj_level 7:    Min 230.7      Max 239.3
k 43 ind 1 Level 5 obj_level 5:    Min 236.6      Max 245.8
k 44 ind 0 Level 2 obj_level 2:    Min 244.7      Max 255.6
Reading my variable:  HGT from GRIB variable: Orography

Reading my variable:  GPH from GRIB variable: Geopotential Height

k 0 ind 44 Level 1000 obj_level 1000:    Min -86.4      Max 184.4
k 1 ind 43 Level 975 obj_level 975:    Min 133.4      Max 398.4
k 2 ind 42 Level 950 obj_level 950:    Min 359.1      Max 614.9
k 3 ind 41 Level 925 obj_level 925:    Min 591.2      Max 835.9
k 4 ind 40 Level 900 obj_level 900:    Min 829.2      Max 1061.8
k 5 ind 39 Level 875 obj_level 875:    Min 1073.1      Max 1297.1
k 6 ind 38 Level 850 obj_level 850:    Min 1323.0      Max 1546.0
k 7 ind 37 Level 825 obj_level 825:    Min 1578.9      Max 1800.5
k 8 ind 36 Level 800 obj_level 800:    Min 1841.5      Max 2061.2
k 9 ind 35 Level 775 obj_level 775:    Min 2111.0      Max 2329.0
k 10 ind 34 Level 750 obj_level 750:    Min

...

k 18 ind 26 Level 550 obj_level 550:    Min 4845.8      Max 5135.4
k 19 ind 25 Level 525 obj_level 525:    Min 5193.8      Max 5508.3
k 20 ind 24 Level 500 obj_level 500:    Min 5554.8      Max 5892.8
k 21 ind 23 Level 475 obj_level 475:    Min 5930.4      Max 6296.4
k 22 ind 22 Level 450 obj_level 450:    Min 6321.0      Max 6717.8
k 23 ind 21 Level 425 obj_level 425:    Min 6729.7      Max 7156.9
k 24 ind 20 Level 400 obj_level 400:    Min 7159.0      Max 7619.0
k 25 ind 19 Level 375 obj_level 375:    Min 7611.9      Max 8110.7
k 26 ind 18 Level 350 obj_level 350:    Min 8092.5      Max 8623.0
k 27 ind 17 Level 325 obj_level 325:    Min 8605.1      Max 9169.8
k 28 ind 16 Level 300 obj_level 300:    Min 9152.5      Max 9750.1
k 29 ind 15 Level 275 obj_level 275:    Min 9740.9      Max 10369.4
k 30 ind 14 Level 250 obj_level 250:    Min 10384.0      Max 11034.6
k 31 ind 13 Level 225 obj_level 225:    Min 11093.5      Max 11752.9
k 32 ind 12 Level 200 obj_level 200:    Min 11883.5      

k 10 ind 34 Level 750 obj_level 750:    Min -18.8      Max 35.3
k 11 ind 33 Level 725 obj_level 725:    Min -20.4      Max 35.2
k 12 ind 32 Level 700 obj_level 700:    Min -19.9      Max 35.9
k 13 ind 31 Level 675 obj_level 675:    Min -19.3      Max 36.6
k 14 ind 30 Level 650 obj_level 650:    Min -19.6      Max 39.0
k 15 ind 29 Level 625 obj_level 625:    Min -20.2      Max 43.6
k 16 ind 28 Level 600 obj_level 600:    Min -20.5      Max 45.4
k 17 ind 27 Level 575 obj_level 575:    Min -19.9      Max 44.9
k 18 ind 26 Level 550 obj_level 550:    Min -19.9      Max 43.3
k 19 ind 25 Level 525 obj_level 525:    Min -20.4      Max 40.6
k 20 ind 24 Level 500 obj_level 500:    Min -19.9      Max 36.7
k 21 ind 23 Level 475 obj_level 475:    Min -19.1      Max 32.9
k 22 ind 22 Level 450 obj_level 450:    Min -20.3      Max 33.3
k 23 ind 21 Level 425 obj_level 425:    Min -22.3      Max 36.0
k 24 ind 20 Level 400 obj_level 400:    Min -24.7      Max 38.0
k 25 ind 19 Level 375 obj_level 375:    

k 36 ind 8 Level 100 obj_level 100:    Min 16437.8      Max 16721.0
k 37 ind 7 Level 70 obj_level 70:    Min 18697.8      Max 18843.1
k 38 ind 6 Level 50 obj_level 50:    Min 20742.1      Max 20952.6
k 39 ind 5 Level 30 obj_level 30:    Min 23970.0      Max 24260.5
k 40 ind 4 Level 20 obj_level 20:    Min 26590.8      Max 26922.8
k 41 ind 3 Level 10 obj_level 10:    Min 31178.2      Max 31555.5
k 42 ind 2 Level 7 obj_level 7:    Min 33591.1      Max 34010.1
k 43 ind 1 Level 5 obj_level 5:    Min 35915.4      Max 36381.7
k 44 ind 0 Level 2 obj_level 2:    Min 42503.4      Max 43121.7
Reading my variable:  W from GRIB variable: Geometric vertical velocity

k 0 ind 44 Level 1000 obj_level 1000:    Min -2.2      Max  1.4
k 1 ind 43 Level 975 obj_level 975:    Min -3.1      Max  1.4
k 2 ind 42 Level 950 obj_level 950:    Min -5.1      Max  2.1
k 3 ind 41 Level 925 obj_level 925:    Min -6.9      Max  3.0
k 4 ind 40 Level 900 obj_level 900:    Min -8.4      Max  4.2
k 5 ind 39 Level 875 obj_

k 28 ind 16 Level 300 obj_level 300:    Min -36.6      Max 36.5
k 29 ind 15 Level 275 obj_level 275:    Min -38.3      Max 40.1
k 30 ind 14 Level 250 obj_level 250:    Min -38.6      Max 44.8
k 31 ind 13 Level 225 obj_level 225:    Min -37.7      Max 44.8
k 32 ind 12 Level 200 obj_level 200:    Min -33.6      Max 39.9
k 33 ind 11 Level 175 obj_level 175:    Min -28.9      Max 36.2
k 34 ind 10 Level 150 obj_level 150:    Min -26.0      Max 30.1
k 35 ind 9 Level 125 obj_level 125:    Min -28.4      Max 23.3
k 36 ind 8 Level 100 obj_level 100:    Min -21.4      Max 20.9
k 37 ind 7 Level 70 obj_level 70:    Min -7.8      Max 10.3
k 38 ind 6 Level 50 obj_level 50:    Min -7.4      Max  8.7
k 39 ind 5 Level 30 obj_level 30:    Min -6.7      Max 10.5
k 40 ind 4 Level 20 obj_level 20:    Min -7.8      Max 10.5
k 41 ind 3 Level 10 obj_level 10:    Min -6.1      Max  7.9
k 42 ind 2 Level 7 obj_level 7:    Min -6.0      Max  7.1
k 43 ind 1 Level 5 obj_level 5:    Min -4.6      Max  6.3
k 44 ind 0

k 11 ind 33 Level 725 obj_level 725:    Min -8.4      Max  9.9
k 12 ind 32 Level 700 obj_level 700:    Min -8.1      Max 10.5
k 13 ind 31 Level 675 obj_level 675:    Min -7.5      Max 11.5
k 14 ind 30 Level 650 obj_level 650:    Min -6.5      Max 12.5
k 15 ind 29 Level 625 obj_level 625:    Min -5.6      Max 13.4
k 16 ind 28 Level 600 obj_level 600:    Min -5.2      Max 14.3
k 17 ind 27 Level 575 obj_level 575:    Min -4.6      Max 15.4
k 18 ind 26 Level 550 obj_level 550:    Min -4.5      Max 16.6
k 19 ind 25 Level 525 obj_level 525:    Min -4.8      Max 17.6
k 20 ind 24 Level 500 obj_level 500:    Min -4.7      Max 18.4
k 21 ind 23 Level 475 obj_level 475:    Min -4.0      Max 18.8
k 22 ind 22 Level 450 obj_level 450:    Min -3.4      Max 18.7
k 23 ind 21 Level 425 obj_level 425:    Min -3.4      Max 19.2
k 24 ind 20 Level 400 obj_level 400:    Min -3.3      Max 20.6
k 25 ind 19 Level 375 obj_level 375:    Min -3.3      Max 22.0
k 26 ind 18 Level 350 obj_level 350:    Min -3.7      M